In [7]:
import pickle
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
#from keras.models import Model, load_model
#from keras.layers import LSTM, Dense, TimeDistributed, Input, Masking, RepeatVector, Bidirectional, Embedding
nltk.download('punkt')
import string
punc = list(string.punctuation)
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from sklearn.metrics import pairwise
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pickle
import csv
import os
from scipy.spatial.distance import cosine
import time
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.feature_extraction.text import CountVectorizer
import torch

In [9]:
## HYPER-PARAMETERS

num_words = 10000
MAX_SEQUENCE_LENGTH = 16
WORD_EMBEDDING_DIM = 200
batch_size = 640
ENCODING_DIM = 500
NUM_DATAPOINT = -1 # All in the file
NUM_ICD_CODES = 2833
NUM_EPOCHS = 2
VALIDATION_SPLIT = 0.2
PATIENCE = 5
TOP_K_PER_SENTENCE = 10
similarity_threshold = 0.7

THRESHOLD = -1 # TODO

ICD_DESC_FILENAME = 'ICD_desc_with_freq.csv'
#DD_FILENAME = 'MIMIC_III_Final_1000_clean_shorted.csv'
#DD_FILENAME = 'MIMIC-III-Final_5000_clean_shortened.csv'
DD_FILENAME = 'MIMIC-III-Final_cleaned.csv'
EMB_FILE = 'PMC-w2v.bin'

#RESULT_FILENAME = 'Experiment Results.xlsx'
#http://yaronvazana.com/2019/09/28/training-an-autoencoder-to-generate-text-embeddings/

In [10]:
#Calculate Sentence Embedding using Average Word Embedding of tokens
def get_sentence_embedding(sent, embedding):
    words = word_tokenize(sent)
    m = len(words)
    total_embedding = 0
    
    #Get wordwise Embeddings as tensors
    sent_embedding = np.array([get_word_embedding(word, embedding) for word in words])
    
    #Get the embedding of sentence by adding embedding tensors
    total_embedding = np.sum(sent_embedding, axis = 0)
    
    try:
        return total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True)
    except:
        #print('sent : ', sent)
        #print('m: ', m)
        return np.array(total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True))

In [11]:
#Calculate Word Embedding
def get_word_embedding(word, embedding):
    try:
        return embedding[word] 
    except:
        return embedding['unk']

In [12]:
#Load Word Embedding (Pub Med W2V)
from gensim.models.keyedvectors import KeyedVectors
word_model = KeyedVectors.load_word2vec_format(EMB_FILE, binary=True)
print('Found %s word vectors.' % len(word_model.index2word))
print(word_model['Cholera'].shape)

Found 2515686 word vectors.
(200,)


In [13]:
word_model['Cholera']/np.linalg.norm(word_model['Cholera'], 2)

array([-0.00481831,  0.13864845, -0.0831728 , -0.00180067, -0.00696073,
       -0.02875327,  0.06748949,  0.081326  ,  0.11438117,  0.00530638,
       -0.05866855,  0.05307228, -0.09064269,  0.03729252, -0.00409504,
        0.02162931,  0.11918882, -0.05017674,  0.03869108,  0.12593974,
       -0.06599744,  0.13589439,  0.02106961, -0.02205387,  0.04005024,
        0.11348598, -0.09267536, -0.04086379, -0.07475508,  0.02114647,
        0.03469999,  0.02900406,  0.06926779,  0.11697073, -0.01642677,
       -0.00433   ,  0.06071504, -0.1460445 , -0.01645673, -0.0205683 ,
        0.01536769,  0.0160432 , -0.00429431,  0.03412509, -0.05015106,
        0.0433115 ,  0.08293187,  0.14879459, -0.0841924 , -0.06157425,
       -0.02435092, -0.05808348, -0.11018413,  0.01283847, -0.07155055,
        0.08904947, -0.0320092 , -0.11904893, -0.01103841,  0.11339661,
        0.01087772, -0.05354536,  0.0755807 ,  0.07982337,  0.11575826,
       -0.1831625 ,  0.01570357, -0.05355467,  0.065675  , -0.05

# ICD File

In [14]:
ICD_desc_filename = ICD_DESC_FILENAME
df_icd = pd.read_csv(ICD_desc_filename, encoding = 'latin1')

df_icd = df_icd[['Code', 'Long Description']]

print(df_icd.shape)

print('\n', df_icd.head())

(6553, 2)

     Code                                   Long Description
0   4019                 Unspecified essential hypertension
1   4280              Congestive heart failure, unspecified
2  42731                                Atrial fibrillation
3  41401  Coronary atherosclerosis of native coronary ar...
4   5849                  Acute kidney failure, unspecified


In [15]:
df_icd_cleaned = df_icd

#Delete df_icd
del df_icd

def clean_icd(x):
    sent = x
    sent_tokenized = word_tokenize(sent)
    
    clean_sent = ''
    
    for token in sent_tokenized:
        if token.lower() not in punc and token.lower() not in stop_words:
            clean_sent = clean_sent + ' ' + token.lower()
    
    x = clean_sent
    
    del clean_sent
    
    return x


df_icd_cleaned['Long Description'] = df_icd_cleaned.apply(lambda x: clean_icd(x['Long Description']), axis = 1)

df_icd_cleaned.head()

,Code,Long Description
0,4019,unspecified essential hypertension
1,4280,congestive heart failure unspecified
2,42731,atrial fibrillation
3,41401,coronary atherosclerosis native coronary artery
4,5849,acute kidney failure unspecified


In [10]:
#Capture the ICD embedding as a numpy matrix
#icd_sent_emb = df_icd_cleaned['Sentence Embedding'][:]
#icd_sent_emb_list = [x for x in icd_sent_emb]

#icd_sent_emb_matrix = np.asarray(icd_sent_emb_list, dtype=np.float32)
#icd_sent_emb_matrix.shape

In [16]:
icd_sentences = list(df_icd_cleaned.head(NUM_ICD_CODES)['Long Description']) # Number of sentences
icd_codes = list(df_icd_cleaned.head(NUM_ICD_CODES)['Code'])
icd_index = list(range(NUM_ICD_CODES))

print(len(icd_sentences), len(icd_codes), len(icd_index))
print("ICD_Sentences = ", len(icd_sentences))

2833 2833 2833
ICD_Sentences =  2833


In [17]:
# 3 Dictionary of ICD index, ICD Code and ICD desc

# index, codes
dict_index_code = dict(zip(icd_index, icd_codes))

# codes, index
dict_code_index = dict(zip(icd_codes, icd_index))

#index, code desc
dict_index_desc = dict(zip(icd_index, icd_sentences))

#icd sentence embedding matrix slice
#icd_sent_emb_matrix = icd_sent_emb_matrix[:NUM_ICD_CODES]

# Cleaned MIMIC-III DD file

In [28]:
DD_filename = DD_FILENAME
df_DD = pd.read_csv(DD_filename)
df_DD = df_DD[0:5000]
df_DD.reset_index(drop=True, inplace= True)
FILESIZE = df_DD.shape[0]
df_DD.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,clean_text,top_ICD
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,"['25013', '3371', '5849', '5780', 'V5867', '25...",admission date discharge date of birth sex nam...,"['25013', '3371', '5849', '5780', 'V5867', '25..."
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,"['53100', '2851', '07054', '5715', '45621', '5...",admission date discharge date of birth sex rec...,"['53100', '2851', '5715', '45621', '53789', '4..."
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,"['49320', '51881', '486', '20300', '2761', '78...",admission date discharge date of birth sex o c...,"['49320', '51881', '486', '20300', '2761', '78..."
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,"['56081', '5570', '9973', '486', '4019']",admission date discharge date of birth sex nam...,"['56081', '5570', '486', '4019']"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,"['41401', '99604', '4142', '25000', '27800', '...",admission date discharge date of birth sex kno...,"['41401', '99604', '4142', '25000', '27800', '..."


In [19]:
FILESIZE

5000

In [26]:
import spacy
from spacy.language import Language

@Language.component("custom_boundary")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '...' or token.text == '.' or token.text =='!' :
            doc[token.i+1].is_sent_start = True
    return doc

custom_nlp = spacy.load('en_core_web_sm')

custom_nlp.add_pipe("custom_boundary", before='parser')

def removingNegativeSentence(text):
    cleanText = []
    negative_token = ["not","absence","no"]
    custom_doc = custom_nlp(text)
    custom_sentences = list(custom_doc.sents)
    for sentence in custom_sentences:
        #print (sentence.text)
        flag  = False
        for token in sentence:
            #print (type(token))
            if token.text in negative_token:
                #Print the Dependant Word to Neg words
                #print (token.text, token.tag_, token.head.text, token.dep_)
                #displacy.serve(sentence, style='dep')
                flag  = True
                break
        if(not flag):
            #print(sentence.text)
            cleanText.append(sentence.text)
    return (' '.join(cleanText))




In [29]:
DD_Data = df_DD[['ICD9_CODE', 'clean_text']]
del df_DD
DD_Data

,ICD9_CODE,clean_text
0,"['25013', '3371', '5849', '5780', 'V5867', '25...",admission date discharge date of birth sex nam...
1,"['53100', '2851', '07054', '5715', '45621', '5...",admission date discharge date of birth sex rec...
2,"['49320', '51881', '486', '20300', '2761', '78...",admission date discharge date of birth sex o c...
3,"['56081', '5570', '9973', '486', '4019']",admission date discharge date of birth sex nam...
4,"['41401', '99604', '4142', '25000', '27800', '...",admission date discharge date of birth sex kno...
...,...,...
4995,"['5724', '5849', '07071', '7863', '5990', '456...",admission date discharge date of birth sex nam...
4996,"['30500', '2762', '28860', '4589']",admission date discharge date of birth sex on ...
4997,"['41401', '3963', '39891', '3970', '42731', '9...",admission date discharge date of birth sex of ...
4998,"['042', '11599', '4280', '2848', '7801', '2761...",admission date discharge date of birth sex com...


In [30]:
#Negation Removal
DD_Data['clean_text'] = DD_Data['clean_text'].apply(lambda x: removingNegativeSentence(x))
DD_Data

,ICD9_CODE,clean_text
0,"['25013', '3371', '5849', '5780', 'V5867', '25...",admission date discharge date of birth sex nam...
1,"['53100', '2851', '07054', '5715', '45621', '5...",pt was in his until about days pta when he int...
2,"['49320', '51881', '486', '20300', '2761', '78...",admission date discharge date of birth sex o c...
3,"['56081', '5570', '9973', '486', '4019']",admission date discharge date of birth sex nam...
4,"['41401', '99604', '4142', '25000', '27800', '...",admission date discharge date of birth sex kno...
...,...,...
4995,"['5724', '5849', '07071', '7863', '5990', '456...",admission date discharge date of birth sex nam...
4996,"['30500', '2762', '28860', '4589']",admission date discharge date of birth sex on ...
4997,"['41401', '3963', '39891', '3970', '42731', '9...",admission date discharge date of birth sex of ...
4998,"['042', '11599', '4280', '2848', '7801', '2761...",admission date discharge date of birth sex com...


In [31]:
Vocab = {}

doc_len = []

for index in range(DD_Data.shape[0]):
    sent = word_tokenize(DD_Data['clean_text'][index])
    
    doc_len.append(len(sent))
    
    for word in sent:
        if word in Vocab.keys():
            Vocab[word]+=1
        else:
            Vocab[word]=1

print('Vocab Size: ', len(Vocab.keys()))
print('Avg Doc Len: ', float(sum(doc_len))/float(len(doc_len)))

Vocab Size:  26953
Avg Doc Len:  1130.3818


In [32]:
del Vocab, doc_len

In [33]:
#Get TF vector using entire corpus
def get_tf(docs):
    
    vectorizer = CountVectorizer()#CountVectorizer(stop_words = 'english')
    X = vectorizer.fit_transform(docs)#(docs['sentence'])

    # place tf values in a pandas data frame
    tf_vecs = pd.DataFrame(X.T.todense(), index = vectorizer.get_feature_names())
    
    #Take sum accross sentences
    freq_vec = tf_vecs.sum(axis = 1)
    freq_vec = pd.DataFrame(freq_vec, index=freq_vec.index, columns=['Frequency'])
    
    return vectorizer, freq_vec

In [34]:
#remove terms not present in freq_vec
def remove_unwanted_words(words, freq_vec):
    
    pruned_words = [x for x in words if x in freq_vec.index]
    
    #for i in words:
    #    if i in freq_vec.index:
    #        pruned_words.append(i)
    
    return list(set(pruned_words))

In [35]:
def get_tf_vector(sent, freq_vec):
    #tokenize sentence
    words = list(set(word_tokenize(sent)))
    
    #remove terms not occuring in freq_vec
    pruned_words = remove_unwanted_words(words, freq_vec)
    
    #get the slice of freq_vec using the remaining terms
    #print(words, pruned_words)
    freq_slice = freq_vec.loc[pruned_words, :]
    
    #L1 normalize to obtain tf vector
    tf_vec = freq_slice/np.linalg.norm(freq_slice, ord = 1)    

    return tf_vec, pruned_words

In [36]:
#calculate the cosine similarity matrix for 2 sentences
def cosine_sim_matrix(s1, s2):
    
    #Get token-wise embedding
    s1_token_emb = np.array([get_word_embedding(word, word_model) for word in s1])
    #Normalize
    s1_token_emb/=np.linalg.norm(s1_token_emb, axis = 1, ord = 2, keepdims=True)
    
    #Get token-wise embedding of icd text
    s2_token_emb = np.array([get_word_embedding(word, word_model) for word in s2])
    #Normalize
    s2_token_emb/=np.linalg.norm(s2_token_emb, axis = 1, ord = 2, keepdims=True)
    
    
    sim_matrix = s1_token_emb.dot(s2_token_emb.T)
    
    return np.clip(sim_matrix, -1.0, 1.0) #Clip the Elements of the matrix

In [37]:
#Reset Debugging Counter
k = 0
#Calculate the Similarity Score between 2 senteces
def RWMS_similarity(sent_1, sent_2, freq_vec):
    global k
    try:
        
        #get tf vectors
        #start = time.time()
        sent1_tf_vec, sent1 = get_tf_vector(sent_1, freq_vec)
        #sent2_tf_vec, sent2 = get_tf_vector(sent_2, freq_vec)
        
        #tokenized sentences
        sent1 = sent1#list(set(word_tokenize(sent_1)))
        sent2 = list(set(word_tokenize(sent_2)))

        #Compute Cosine Similarity Matrix
        sim_mat = cosine_sim_matrix(sent1, sent2)
        #print('Sim_mat:\n', sim_mat)
        
        #Initialise Similarity values:
        W_s_t = 0
        #W_t_s = 0

        #for S->t (icd_text -> dd text)
        max_cosine_vec = sim_mat.max(axis = 1)
        #print('Sim_mat:\n', sim_mat)
        
        # Take Elementwise product of max cos similarity vector and tf vector
        sim_vec = sent1_tf_vec.to_numpy().transpose() * max_cosine_vec
        
        #Take sum of all elements of sim_vec to obtain final similarity
        W_s_t = np.sum(sim_vec)
        
        del sim_mat, sent1_tf_vec, sent1, sent2
        
        #for t->S
        #for j in range(len(sent2)):
        #    w2 = sent2[j]
        #    sim = 0
        #    if w2 in density_tfidf.index:
        #        #index of word with max cosine similarity from sent1
        #        max_j = np.argmax(sim_mat.transpose()[j])

        #        #multiply the tfidf value of current word of sent2 with cosine similarity of the most similar from sent1
        #        sim = sim_mat.transpose()[j][max_j]*density_tfidf['s2_tfidf'][w2]
        #    else:
        #        sim = 0
        #    W_t_s+=sim
        
        
        #k = k + 1
        #print('k: ', k)

        return W_s_t#, sim_mat, max_cosine_vec, sent1_tf_vec, sent1, sent2   #min(W_s_t, W_t_s)
    except:
        #print('sent_1: ', sent_1)
        sent_1_emb = get_sentence_embedding(sent1, word_model)
        #print('sent_2: ', sent_2)
        sent_2_emb = get_sentence_embedding(sent2, word_model)
        
        #k = k + 1
        #print('k: ', k)
        
        return sent_1_emb.dot(sent_2_emb.T)#, 0, 0, 0, 0, 0#cosine_sim(sent_1_emb, sent_2_emb)

In [38]:
#Convert ICD Indices to hot vectors of length: NUM_ICD_CODE
def generate_binary(icd_index_list, total_icd = NUM_ICD_CODES):
    binary_icd = np.zeros(total_icd)
    for x in icd_index_list:
        binary_icd[x] = 1
        
    return binary_icd

In [39]:
#Calculate Specificity

def calc_specificity(pred_vec, act_vec):
    m = len(pred_vec)
    
    specificity = []
    
    for i in range(m):
        pred, act = pred_vec[i], act_vec[i]
        dict_XY = {'pred':pred, 'act':act}
        res = pd.DataFrame(dict_XY, columns=['pred', 'act'])
        res['Sum'] = res['pred'] + res['act']
        
        pred_p = res[res.pred == 1]
        pred_n = res[res.act == 1]
        
        tp = len(res[res.Sum == 2])
        tn = len(res[res.Sum == 0]) 
        
        fp = len(pred_p[pred_p.act == 0])
        fn = len(pred_n[pred_n.act == 1])
        
        #Calculate specificity for this example:
        spec = tn/(tn + fp)
        
        specificity.append(spec)
    
    return sum(specificity) / float(m)
        

In [40]:
#Predict ICD 9 Codes

#construct the Term Frequency Vector using just the icd corpus
c_vectorizer, freq_vec = get_tf(df_icd_cleaned['Long Description'])

start_time = time.time()

#Calculate the Similarity of each document with each icd text
X = DD_Data['clean_text']#[0]
Y = list(DD_Data['ICD9_CODE'])
m = len(Y)

sim = []

for i in range(m):
    text = X[i]
    #Modified#Calculate cosine similarity of sentence with icd sentences
    sim.append(np.array([RWMS_similarity(icd_text, text, freq_vec) for icd_text in icd_sentences]))#cosine_sim1_matrix(sent_emb)
    
    #reset debugging counter
    #global k
    #k = 0
    
    if (i+1)%10 == 0:
        print(str(i+1) + '/' + str(m) + ' records done.')
    
    if (i+1)%100 == 0:
        print('\nTime elapsed: ', (time.time() - start_time)/60.0, ' minutes')

    if (i+1)%1000 == 0:
        sim_df = pd.DataFrame(sim, dtype = float)
        sim_df.to_csv('Similarity_Data/Optimized_NP_RWMS_Negation_similarity_data_' + str(i+1) + '_records.csv')
        #sim_df.to_csv('Optimized_NP_RWMS_similarity_data_' + (i+1) + '_records.csv')

time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')

#Save the similarity scores in an excel file
print('Saving Similarity scores in file......')
sim_df = pd.DataFrame(sim, dtype = float)
sim_df.to_csv('Similarity_Data/Optimized_NP_RWMS_Negation_similarity_data_' + str(m) + '_records.csv')
#sim_df.to_csv('Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')
print('Similarity Scores Saved!')

10/5000 records done.
20/5000 records done.
30/5000 records done.
40/5000 records done.
50/5000 records done.
60/5000 records done.
70/5000 records done.
80/5000 records done.
90/5000 records done.
100/5000 records done.

Time elapsed:  67.83916195233662  minutes
110/5000 records done.
120/5000 records done.
130/5000 records done.
140/5000 records done.
150/5000 records done.
160/5000 records done.
170/5000 records done.
180/5000 records done.
190/5000 records done.
200/5000 records done.

Time elapsed:  142.82773968378703  minutes
210/5000 records done.
220/5000 records done.
230/5000 records done.
240/5000 records done.
250/5000 records done.
260/5000 records done.
270/5000 records done.
280/5000 records done.
290/5000 records done.
300/5000 records done.

Time elapsed:  213.87786363363267  minutes
310/5000 records done.
320/5000 records done.
330/5000 records done.
340/5000 records done.
350/5000 records done.
360/5000 records done.
370/5000 records done.
380/5000 records done.
390/

2950/5000 records done.
2960/5000 records done.
2970/5000 records done.
2980/5000 records done.
2990/5000 records done.
3000/5000 records done.

Time elapsed:  2181.7674114545184  minutes
3010/5000 records done.
3020/5000 records done.
3030/5000 records done.
3040/5000 records done.
3050/5000 records done.
3060/5000 records done.
3070/5000 records done.
3080/5000 records done.
3090/5000 records done.
3100/5000 records done.

Time elapsed:  2246.06022768418  minutes
3110/5000 records done.
3120/5000 records done.
3130/5000 records done.
3140/5000 records done.
3150/5000 records done.
3160/5000 records done.
3170/5000 records done.
3180/5000 records done.
3190/5000 records done.
3200/5000 records done.

Time elapsed:  2312.5769924998285  minutes
3210/5000 records done.
3220/5000 records done.
3230/5000 records done.
3240/5000 records done.
3250/5000 records done.
3260/5000 records done.
3270/5000 records done.
3280/5000 records done.
3290/5000 records done.
3300/5000 records done.

Time 

In [41]:
#Load Similarity Data
sim = pd.read_csv('Similarity_Data/Optimized_NP_RWMS_Negation_similarity_data_' + str(m) + '_records.csv')
#sim = pd.read_csv('Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')

#Drop first column
sim = sim.drop(['Unnamed: 0'], axis = 1)

#Convert DataFrame to List
sim = np.array(sim.values.tolist())

In [42]:
sim_df.shape

(5000, 2833)

In [43]:
len(sim)

5000

In [44]:
#Save Predictions and Corresponding Scores per threshold

#Initialise Similarity Result Dataframe
results_sim = pd.DataFrame(columns = ['Actual Code', 
                                  'Predicted Codes',
                                  'Similarity Codes'])


threshold_list = [0.6, 0.7, 0.8, 0.9]

#Initialize Excel
#writer = pd.ExcelWriter('Similarity_Data/Code and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records.xlsx', engine='xlsxwriter')

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)

    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])

    data_input_format = []

    i = 0
    for i in range(m):
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_codes = [str(y) for y in Y[i][2:-2].split("', '") if y in icd_codes]#To get rid of '' and ,
        act_codes_string = ', '.join(act_codes)

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh


        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred_indices = np.nonzero(sim_post_threshold)[0].tolist()

        #Get codes from indices
        pred_codes = [str(dict_index_code[p]) for p in pred_indices]
        #Convert to Comma Separated String
        pred_codes_string = ', '.join(pred_codes)
        #print([pred_codes_string])

        #Corresponding Similarity Scores of List of Indices
        sim_scores = [str(sim[i][j]) for j in pred_indices]
        #Convert to Comma Separated String
        sim_scores_string = ', '.join(sim_scores)
        #print([sim_scores_string])


        data_input = [act_codes_string, pred_codes_string, sim_scores_string]

        #Append
        data_input_format.append(data_input)

    df = pd.DataFrame(data_input_format, columns = ['Actual Codes', 'Predicted Codes', 'Similarity Scores'])
    df.to_csv('Similarity_Data/Negation - Code and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')
    #df.to_csv('NP - Predicted Codes and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')


Commenced for Similarity Threshold: 0.6

Commenced for Similarity Threshold: 0.7

Commenced for Similarity Threshold: 0.8

Commenced for Similarity Threshold: 0.9


In [45]:
#Calculate Metrics

#Initialize Results Dataframe which will store the relevant metrics for each threshold
results = pd.DataFrame(columns = ['Number of Observations', 
                                  'Number of ICD Codes',
                                  'Threshold',
                                  'Precision', 
                                  'Recall', 
                                  'F1-Score', 
                                  'Specificity'])

#Compute Accuracy metrics based on thresholding        
#threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
threshold_list = [0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.4]
#threshold_list = [1.0, 5.0 10.0, 15.0, 20.0]

start_time = time.time()

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)
    
    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])
    
    predicted_codes_index = []
    actual_codes_index = []

    #m = 1#len(Y)

    for i in range(m):
        #text = X#[i]
        #print(i)
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_icd_indices = [dict_code_index[y] for y in Y[i][2:-2].split("', '") if y in icd_codes] #To get rid of '' and ,

        #Initialize Prediction list. This will contain indices of similar icd codes
        pred = []

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh

        
        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred.extend(np.nonzero(sim_post_threshold)[0].tolist())

        #Consider the unique indices
        pred = set(pred)

        #Convert list of indices to hot vector
        pred_icd_vector = generate_binary(pred)
        act_icd_vector = generate_binary(act_icd_indices)

        #Append
        predicted_codes_index.append(pred_icd_vector)
        actual_codes_index.append(act_icd_vector)


        #if i%100 == 0 and i != 0:
        #    print(i, ' sentences done.')

    #Calculate Precision, Recall, F1 Score for this threshold
    precision, recall, f1_score, support = precision_recall_fscore_support(actual_codes_index, predicted_codes_index, average = 'micro')
    specificity = calc_specificity(predicted_codes_index, actual_codes_index)

    #Add to results Dataframe:
    results = results.append({'Number of Observations': m, 
                    'Number of ICD Codes': NUM_ICD_CODES, 
                    'Threshold': thresh,
                    'Precision': precision, 
                    'Recall': recall, 
                    'F1-Score': f1_score, 
                    'Specificity': specificity}, ignore_index=True)

    print('\nPrecision: ', precision, ' Recall: ', recall, ' F1-Score: ', f1_score, ' Specificity:', specificity)


time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')    
    
#Print Results
RESULT_FILENAME = 'Results/Experiment_Results_RWMS_Negation_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
#RESULT_FILENAME = 'Experiment_Results_NP_RWMS_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
results.to_excel(RESULT_FILENAME) 
print('Results Saved!')
results


Commenced for Similarity Threshold: 0.6

Precision:  0.00584824536306984  Recall:  0.5570257311225797  F1-Score:  0.011574964504443624  Specificity: 0.633190276736831

Commenced for Similarity Threshold: 0.7

Precision:  0.008038848364757562  Recall:  0.40805607408220784  F1-Score:  0.01576707970659386  Specificity: 0.8049285307275535

Commenced for Similarity Threshold: 0.8

Precision:  0.011413698740507768  Recall:  0.2903627246440467  F1-Score:  0.021964026396700826  Specificity: 0.902557137789861

Commenced for Similarity Threshold: 0.9

Precision:  0.014884067005280848  Recall:  0.1918304600856484  F1-Score:  0.02762473892618776  Specificity: 0.9508049671668832

Time elapsed:  1.4645623167355855  minutes
Results Saved!


,Number of Observations,Number of ICD Codes,Threshold,Precision,Recall,F1-Score,Specificity
0,5000.0,2833.0,0.6,0.005848,0.557026,0.011575,0.633190
1,5000.0,2833.0,0.7,0.008039,0.408056,0.015767,0.804929
2,5000.0,2833.0,0.8,0.011414,0.290363,0.021964,0.902557
3,5000.0,2833.0,0.9,0.014884,0.191830,0.027625,0.950805
